In [ ]:
!pip install git+https://github.com/openai/CLIP.git
!pip install torch torchvision
!pip install matplotlib
!pip install tqdm
!pip install lightning

In [2]:
import os
import sys

module_path = os.path.join(os.getcwd(), '../src')
sys.path.append(module_path)

import torch
from gan_t2i.models.CLIP import CLIPModel
from gan_t2i.utils.model_loading import download_CLIP_model , CLIP_DATASETS



from gan_t2i.datasets.DatasetFactory import DatasetFactory
import torchvision.transforms as transforms
import clip
from PIL import Image

In [39]:
from gan_t2i.models.GAN import WGAN
#from gan_t2i.models.GAN_INT_CLS import GAN_INT_CLS
#from gan_t2i.models.GAN_Vanilla import GAN

In [ ]:
print(torch.cuda.is_available())

# Loading data from checkpoints

In [ ]:
checkpoints_path = download_CLIP_model(CLIP_DATASETS.FLOWERS)

In [ ]:
clip_model = CLIPModel.load(checkpoints_path)

------------------------------------

In [7]:
from torch.utils.data import DataLoader , SubsetRandomSampler

# Loading dataset

In [8]:
""" Text Transformation

You need to tokenize your text before passing it to the model.
"""
def tokenize_text(text):

    # Try except is needed due to error thrown by CLIP model that limit the context size
    # to 77 tokens so we need to split the text in smaller chunks and keep only a small portion
    # of it if the text is too long
    try:
        return clip.tokenize([text])[0]
    except:
        return clip.tokenize([text.split(".")[0]])[0]


In [9]:
""" Image transformations """
transform_img = transforms.Compose([
    transforms.Resize(64, interpolation=Image.BICUBIC),
    transforms.CenterCrop(64),
    transforms.ToTensor(),

    # Previously computed mean and std
    transforms.Normalize([0.4355, 0.3777, 0.2879], [0.2571, 0.2028, 0.2101])
])

In [ ]:
dataset = DatasetFactory.Flowers(os.path.join(os.getcwd(), "..", "data"), transform_img=transform_img, transform_caption=tokenize_text)

In [ ]:
# Create train, validation and test set     NOTE: We are using small subset of the dataset for this example
#
# SMALL DATASET                                           This may and will cause overfitting this is only a demo
train_size = int(0.8 * len(dataset))
val_size = int(0.01 * len(dataset))
test_size = int(0.02 * len(dataset))

# BIG DATASET
#train_size = int(0.8 * len(dataset))
#val_size = int(0.4 * len(dataset))
#test_size = int(0.2 * len(dataset))

print("train_size: {} , val_size: {} , test_size: {}".format(train_size,val_size,test_size))

# Cration of train, validation and test set indices and samplers
train_indices = list(range(train_size))
val_indices = list(range(train_size, train_size + val_size))
test_indices = list(range(train_size + val_size, train_size + val_size + test_size))

train_sampler = SubsetRandomSampler(train_indices)
val_sampler = SubsetRandomSampler(val_indices)
test_sampler = SubsetRandomSampler(test_indices)

batch_size=256
# Creation of train, validation and test dataloaders
train_loader = DataLoader(dataset, batch_size=batch_size, sampler=train_sampler, pin_memory=True)
val_loader = DataLoader(dataset, batch_size=batch_size, sampler=val_sampler, pin_memory=True)
test_loader = DataLoader(dataset, batch_size=batch_size, sampler=test_sampler, pin_memory=True)

In [ ]:
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
import torch

def show_images(images, num_images=4):
    """
    Visualizza un set di immagini.

    Parameters:
        images (torch.Tensor): Le immagini da visualizzare. Devono avere forma (N, C, H, W).
        num_images (int): Il numero di immagini da visualizzare.
    """
    # Seleziona solo il numero di immagini specificato
    images = images[:num_images]

    # Cambia l'ordine dei canali da CHW a HWC per la visualizzazione
    images = images.permute(0, 2, 3, 1)  # (N, H, W, C)

    # Normalizza le immagini nel range [0, 1]
    images = (images - images.min()) / (images.max() - images.min())

    # Crea una figura per visualizzare le immagini
    fig, axs = plt.subplots(1, num_images, figsize=(num_images * 3, 3))

    for i in range(num_images):
        # Mostra l'immagine
        axs[i].imshow(images[i].cpu().detach().numpy())
        axs[i].axis('off')  # Nasconde gli assi per una visualizzazione più pulita

    plt.show()


for images, captions, _ in test_loader:

    print("Min values : {}  | Max Values : {} | type : {}".format(images.min().item(),images.max().item(),images.dtype))
    show_images(images, num_images=4)

    # Rompe il ciclo dopo il primo batch per facilitare la visualizzazione
    break


---------------------------------------

# Creating the GAN model and training it

- Get the image and text embedding

In [ ]:
# Per ottenere l'ultimo livello
dim_img_size , dim_text_size = clip_model.get_output_dimensions()
print("Cosa ritorna CLIP due vettori da: :", dim_img_size[1] , dim_text_size[1])

In [ ]:
embedding_size = dim_text_size[1]                     # dimensione di output quando codifico il testo con CLIP
p_emb_dim = 128
WGAN_model = WGAN(clip_model,embedding_size,p_emb_dim)

print("Alla classe WGAN viene passato il modello di CLIP gia addestrato da prima sul dataset Flower e due dimensioni (output di clip = {}, proiezione = {})".format(embedding_size,p_emb_dim))


In [ ]:
checkpoints_path = os.path.join(os.getcwd(), "checkpoints_WGAN2")

WGAN_model.fit(
    train_dataloader = train_loader,
    val_dataloader = val_loader,
    num_epochs = 100,
    save_path=checkpoints_path
)

---------------

# Load trained Model and predict an image based on the caption

In [ ]:
checkpoints_full_path = os.path.join(os.getcwd(), "checkpoints_WGAN2\WGAN_epoch-2.pt") # choose your epoch
aux_model = WGAN(clip_model,embedding_size,p_emb_dim)
WGAN_model = aux_model.load(model_pt_filepath=checkpoints_full_path)

In [ ]:
#for (images, captions, _) in test_loader:
for (images, captions, _) in train_loader:
    generated_images = WGAN_model.predict(captions)
    print("generated_images shape {} ".format(generated_images.shape))  # [256, 3, 224, 224]

    print("Min values : {}  | Max Values : {} | type : {}".format(generated_images.min().item(),generated_images.max().item(),generated_images.dtype))
    show_images(generated_images,num_images=4)

    break

------------------

# Load model from an epoch and start training from there

In [ ]:
checkpoints_full_path = os.path.join(os.getcwd(), "checkpoints_WGAN\WGAN_epoch-2.pt")
aux_model = WGAN(clip_model,embedding_size,p_emb_dim)
WGAN_model = aux_model.load(model_pt_filepath=checkpoints_full_path)

In [ ]:
checkpoints_path = os.path.join(os.getcwd(), "checkpoints_WGAN2")

WGAN_model.fit(
    train_dataloader = train_loader,
    val_dataloader = val_loader,
    num_epochs = 2,
    save_path=checkpoints_path,
    starting_epoch = 2
)